In [1]:
import numpy as np
from os import walk
import re
import requests
from bs4 import BeautifulSoup,Comment
import re
import pandas as pd

In [2]:
import pandas as pd
import os
import shutil

In [3]:
FOLDER_PATH = './dataset/foodnetwork/'
# test_file = '0a5ae5d3ccf0e0012ccb68716c9af39c.html'

In [4]:
def normalize_string(string):
    return re.sub(
        r'\s+', ' ',
        string.replace(
            '\xa0', ' ').replace(  # &nbsp;
            '\n', ' ').replace(
            '\t', ' ').strip()
    )

In [5]:
TIME_REGEX = re.compile(
    r'(\D*(?P<hours>\d+)\s*(hours|hrs|hr|h|Hours|H))?(\D*(?P<minutes>\d+)\s*(minutes|mins|min|m|Minutes|M))?')

def get_minutes(element):
    try:
        tstring = element.get_text()
        if '-' in tstring:
            tstring = tstring.split('-')[1]  # sometimes formats are like this: '12-15 minutes'
        matched = TIME_REGEX.search(tstring)

        minutes = int(matched.groupdict().get('minutes') or 0)
        minutes += 60 * int(matched.groupdict().get('hours') or 0)

        return minutes
    except AttributeError:  # if dom_element not found or no matched
        return 0

In [6]:
done_list = []

In [ ]:
foodnetwork_dict = []

In [ ]:
for root, _, files in walk(FOLDER_PATH):
    for name in files:
        if name in done_list:
            print('The file has been scrapped.')
            continue
        soup_i = BeautifulSoup(open(FOLDER_PATH + name, 'rb'), 'html.parser')
        if soup_i.findAll('a'):    #make sure there is no blank HTML
            if ('http://www.foodnetwork.com/recipes/' in soup_i.text and soup_i.find("meta", property="og:title")): # make sure the page is for recipe
                
                title = soup_i.find("meta", property="og:title")['content']
                
                html = name
                
                if soup_i.find('div',class_="rm-block lead hreview-aggregate review"):
                    rating_tmp = re.findall(r'Rated (\d+) stars', soup_i.find('div',class_="rm-block lead hreview-aggregate review").text)
                else:
                    rating_tmp = []
                    
                if rating_tmp:
                    Rating = float(re.findall(r'Rated (\d+) stars', soup_i.find('div',class_="rm-block lead hreview-aggregate review").text)[0])
                else:
                    Rating = np.nan

                if soup_i.findAll('span', {'class':"lgbtn-text"}):
                    review_tmp = re.findall('(\d+)',soup_i.findAll('span', {'class':"lgbtn-text"})[1].text)
                else:
                    review_tmp = []
                
                if review_tmp:
                    Review_cnt = int(re.findall('(\d+)',soup_i.findAll('span', {'class':"lgbtn-text"})[1].text)[0])
                else:
                    Review_cnt = np.nan
                
                if soup_i.find('dd',class_="head duration clrfix"):
                    Total_time = get_minutes(soup_i.find('dd',class_="head duration clrfix"))
                else:
                    Total_time = np.nan
                    
                if soup_i.find('dd',class_="cookTime clrfix"):
                    cook_time = get_minutes(soup_i.find('dd',class_="cookTime clrfix"))
                else:
                    cook_time = []
                
                if soup_i.find('dd',class_="prepTime clrfix"):
                    prep_time = get_minutes(soup_i.find('dd',class_="prepTime clrfix"))
                else:
                    prep_time = []
                    
                
                ingredients = soup_i.findAll('li', {'class': 'ingredient'})
                if ingredients:
                    ingredient = [normalize_string(ingredient.get_text())for ingredient in ingredients]
                else:
                    ingredient = np.nan
                    
                directions = soup_i.findAll('div', class_="instructions")
                if directions:
                    direction = [normalize_string(direction.get_text()) for direction in directions]
                else:
                    direction = np.nan
                    
                cuisine_try = re.search(r'(Cuisine)\W*(\w*)',soup_i.text).group(2)
                if cuisine_try == 'mdManager':
                    cuisine = np.nan
                else:
                    cuisine = cuisine_try

                tmp_dict = {'Title':title,'Rating':Rating,'Review_cnt':Review_cnt,'Total_time':Total_time,\
                            'Prep_time':prep_time,'Cook_time':cook_time,'ingredient':ingredient,\
                            'direction':direction,'html':html,'Cuisine':cuisine}
                
                foodnetwork_dict.append(tmp_dict)
                
                print(count, name)
            else:
                null_list.append(name)
                print(count, name, 'not recipe pages')
        else:
            null_list.append(name)
            print(count, name, 'null pages')
        done_list.append(name)
        count+=1

In [ ]:
foodnetwork_withCuisine = pd.DataFrame(foodnetwork_dict)
foodnetwork_withCuisine.to_csv('./SavedList/foodnetwork_withCuisine.csv')

In [150]:
foodnetwork_withCuisine = pd.read_csv('./SavedList/foodnetwork_withCuisine.csv')

In [151]:
foodnetwork_withCuisine.Title.count()

7577

In [152]:
foodnetwork = foodnetwork_withCuisine.drop_duplicates(subset=['Title']).copy()

In [156]:
foodnetwork.drop(columns=['Unnamed: 0','html'],inplace=True)

In [158]:
cols = ['Title','Cuisine','Total_time','Cook_time','Prep_time','Rating','Review_cnt','ingredient','direction']

In [161]:
foodnetwork = foodnetwork[cols]
foodnetwork

,Title,Cuisine,Total_time,Cook_time,Prep_time,Rating,Review_cnt,ingredient,direction
0,French Onion Dip,NaN,150.0,40,20,5.0,52.0,"['2 tablespoons canola oil', '1 tablespoon but...","[""In a large saute pan over medium high heat, ..."
1,Swedish Meatballs,European,55.0,25,30,5.0,271.0,"['2 slices fresh white bread', '1/4 cup milk',...",['Preheat oven to 200 degrees F. Tear the brea...
2,Sauteed Yellow Squash,American,24.0,14,10,5.0,35.0,"['1 tablespoon extra-virgin olive oil', '2 tab...",['Preheat skillet over medium high heat. Add o...
3,Butternut Squash Lasagna,European,130.0,85,30,5.0,385.0,"['1 tablespoon olive oil', '1 (1 1/2 to 2-poun...",['Heat the oil in a heavy large skillet over m...
4,Emeril's Essence,American,0.0,[],[],5.0,74.0,"['2 1/2 tablespoons paprika', '2 tablespoons s...",['Combine all ingredients thoroughly.']
5,Cucumber Salad,American,10.0,[],10,4.0,1.0,"['2 hothouse cucumbers, peeled and sliced', '1...",['Simply toss together.']
6,Scallops Provencal,European,15.0,10,5,5.0,337.0,"['1 pound fresh bay or sea scallops', 'Kosher ...","[""If you're using bay scallops, keep them whol..."
7,Baked Cheese Grits,American,70.0,60,5,5.0,13.0,"['1/4 cup plus 2 tablespoons butter', '4 cups ...",['Preheat oven to 350 degrees F. Grease a 2-qu...
8,American Macaroni Salad,American,30.0,10,20,5.0,223.0,"['2 cups dry elbow macaroni, cooked, rinsed, a...","['In a large bowl combine the macaroni, celery..."
9,Heirloom Carrot Spoon Cake,American,273.0,270,3,4.0,32.0,['1 (18.25-ounce) box spice cake mix (recommen...,['Spray a 5-quart slow cooker with butter flav...


In [162]:
foodnetwork.to_csv('./SavedList/foodnetwork_final.csv',index=False)

In [96]:
with open('./foodnetwork_not_recipe.txt', 'w') as f:
    for item in null_list:
        f.write("%s\n" % item)

In [164]:
PATH = '/Users/yan/Documents/GitHub/ADA/Project/dataset/'

PATHSRC = PATH+'foodnetwork/'

PATHDES = PATH+'nullhtml/'

# move html from all recipePages to corresponding folder
for name in null_list:
    src = PATHSRC+name
    #print(src)
    des = PATHDES
    shutil.move(src,des)